In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

/Users/omer/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/omer/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/omer/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/omer/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/f

In [2]:
#read data & formatting

Xraw = pd.read_csv('/Users/omer/Documents/YK Data/X.csv', index_col=0, dtype="category")
Yraw = pd.read_csv('/Users/omer/Documents/YK Data/Y.csv', index_col=0)
X = Xraw.to_numpy()
Y = Yraw.to_numpy()

In [3]:
#split train and test sets

train_size = int(len(X) * 0.8) 
test_size  = len(X) - train_size 
X_train, X_test = X[0:train_size,:], X[train_size:len(X), :]
Y_train, Y_test = Y[0:train_size,:], Y[train_size:len(Y), :]

In [4]:
#calculate class weight to accomaodate for imbalance

Ydf = pd.DataFrame(Y_train,index=Y_train[:,0])
y_classes = Ydf.idxmax(1)
le = LabelEncoder()
le.fit(list(y_classes))
y_integers = le.transform(list(y_classes))
labels_and_integers = dict(zip(y_classes, y_integers))
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
sample_weights = compute_sample_weight('balanced', y_integers)

class_weights_dict = dict(zip(le.transform(list(le.classes_)), class_weights))

In [5]:
#learn embeddings -- train

model_embed = Sequential()
model_embed.add(Embedding(140, 1, input_length=1578))
model_embed.compile('Adam', 'categorical_crossentropy')
X_train_embedded = model_embed.predict(X_train)

In [6]:
#learn embeddings -- test

model_embed = Sequential()
model_embed.add(Embedding(140, 1, input_length=1578))
model_embed.compile('Adam', 'categorical_crossentropy')
X_test_embedded = model_embed.predict(X_test)

In [7]:
#reshape for LSTM

X_train3 = np.reshape(X_train_embedded,(X_train_embedded.shape[0], 1, X_train.shape[1]))
X_test3  = np.reshape(X_test_embedded,(X_test_embedded.shape[0], 1, X_test.shape[1]))

In [13]:
#model

model_lstm = Sequential()

model_lstm.add(LSTM(units=256, input_shape = (1,1578), dropout = 0.3, recurrent_dropout = 0.3))
model_lstm.add(Dense(256, activation = 'relu'))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(53, activation = 'softmax'))
model_lstm.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=[tf.keras.metrics.Recall()]
)

In [14]:
#fit the model model

h = model_lstm.fit(X_train3, Y_train, validation_split=0.2, verbose=1, epochs=5, batch_size=10, class_weight=class_weights)

Train on 6467 samples, validate on 1617 samples
Epoch 1/5
6467/6467 [==============================] - 35s 5ms/step - loss: 1.3142 - recall_1: 0.5695 - val_loss: 1.2093 - val_recall_1: 0.6209
Epoch 2/5
6467/6467 [==============================] - 36s 6ms/step - loss: 1.1445 - recall_1: 0.6197 - val_loss: 1.1772 - val_recall_1: 0.6257
Epoch 3/5
6467/6467 [==============================] - 21s 3ms/step - loss: 1.1175 - recall_1: 0.6294 - val_loss: 1.1808 - val_recall_1: 0.6290
Epoch 4/5
6467/6467 [==============================] - 22s 3ms/step - loss: 1.1030 - recall_1: 0.6297 - val_loss: 1.1766 - val_recall_1: 0.6311
Epoch 5/5
6467/6467 [==============================] - 23s 4ms/step - loss: 1.0929 - recall_1: 0.6328 - val_loss: 1.1583 - val_recall_1: 0.6338


In [458]:
#make predictions

train_predict = model_lstm.predict(X_train3)
test_predict  = model_lstm.predict(X_test3) 

In [544]:
print (Y_train[[23]])

[[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [492]:
print (train_predict[[990]].round(decimals=3))

[[0.051 0.173 0.08  0.325 0.068 0.02  0.034 0.002 0.    0.    0.007 0.033
  0.005 0.    0.    0.033 0.006 0.003 0.02  0.    0.023 0.01  0.    0.
  0.    0.04  0.001 0.    0.007 0.    0.001 0.    0.007 0.    0.003 0.
  0.002 0.    0.009 0.002 0.    0.    0.007 0.019 0.    0.    0.001 0.003
  0.004 0.    0.    0.    0.001]]


In [547]:
print (len(np.unique(y_integers)))

49


In [362]:
print (Y_train[[6]])

[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [545]:
print (sample_weights[[23]])

[1.60174361]


In [454]:
print(a)

Tensor("Cast_2:0", shape=(8084,), dtype=float32)


In [461]:
test_predict[[123]]

array([[4.1132964e-05, 5.0477274e-06, 2.1648123e-08, 8.1232523e-05,
        2.6170997e-02, 9.7367913e-01, 6.4839227e-09, 1.4643424e-10,
        2.2489919e-09, 1.7345250e-09, 1.3180698e-10, 1.6093573e-07,
        3.5240680e-07, 8.5104715e-11, 2.7130259e-08, 1.9890789e-09,
        2.0272937e-12, 1.3184450e-09, 5.3039451e-10, 4.7114646e-10,
        7.8823323e-11, 3.7944175e-12, 9.7913699e-10, 1.4037573e-07,
        5.0944413e-07, 3.0060676e-07, 5.2274172e-09, 5.9216645e-06,
        1.5515261e-10, 5.2484830e-07, 3.5815390e-10, 1.4395304e-09,
        8.1494314e-09, 3.6166558e-08, 1.6632973e-10, 3.2424794e-09,
        4.0044371e-10, 1.2779539e-09, 1.1972772e-09, 1.6128829e-06,
        4.6798448e-08, 2.1439709e-08, 1.7581948e-12, 6.9597930e-12,
        4.2803094e-09, 1.5456793e-08, 3.3644594e-11, 1.4472389e-09,
        1.7857438e-07, 1.0669618e-06, 7.6953829e-06, 3.7362572e-06,
        3.6850237e-10]], dtype=float32)